In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Get y

See https://tc02d-dev.c3dti.ai/jupyter/notebooks/research_teams/gordon/accessing-data/Fetch3HourlyAODData.ipynb.

In [2]:
def get_response(gstp):
    # filter
    s3haodFilter = c3.Filter().eq("geoSurfaceTimePoint", gstp)
    
    # collect objects
    data = c3.Simulation3HourlyAODOutput.fetch({"filter": s3haodFilter})

    # cast it into pandas
    data = data.objs.toJson()
    return np.array(pd.DataFrame(data).loc[:, 'dust':'insolubleAitkenMode'].sum(axis=1))

In [3]:
def get_average_response(
    time1="2017-07-01T00:00:00",
    time2="2017-07-31T23:59:59",
    lat1 = 0.0,
    lat2 = 1.0,
    lon1 = 0.0,
    lon2 = 1.0
):
    gstpFilter = c3.Filter() \
        .ge("latitude", lat1) \
        .and_().le("latitude", lat2) \
        .and_().ge("longitude", lon1) \
        .and_().le("longitude", lon2) \
        .and_().ge("time", time1) \
        .and_().le("time", time2)
    gstps = c3.GeoSurfaceTimePoint.fetch({"filter": gstpFilter})
    
    each_response = [get_response(gstp) for gstp in gstps.objs]
    response_full = [len(resp)==221 for resp in each_response]
    
    my_dict = dict(zip(
        [gstps.objs[k].id for k in range(len(gstps.objs)) if response_full[k]],
        [each_response[k] for k in range(len(gstps.objs)) if response_full[k]]
    ))
    
    return pd.DataFrame(my_dict).mean(axis=1)

# Get x

See https://tc02d-dev.c3dti.ai/jupyter/notebooks/research_teams/gordon/accessing-data/ensemble-members.ipynb.

In [ ]:
def get_inputs(
    myFeaturesNames = [
        "bparam",
        "acure_autoconv_exp_nd",
        "a_ent_1_rp",
        "acure_dry_dep_acc",
        "acure_dry_dep_ait",
        "acure_dry_dep_so2",
        "acure_bvoc_soa",
        "acure_dms",
        "acure_bl_nuc",
        "acure_ait_width",
        "acure_cloud_ph",
        "dbsdtbs_turb_0",
        "acure_bc_ri",
        "acure_sea_spray",
        "acure_carb_bb_diam",
        "acure_anth_so2_r",
        "acure_prim_so4_diam"
    ]
):
    # Obtain a comma-separated table of simulation ensemble members' parameter combinations
    csv_table_metric = c3.SimulationModelParameters.fetch().objs.toJson()

    # Save this table instead as a pandas dataframe (221 ensemble members x 64 parameters)
    dfparams = pd.DataFrame(csv_table_metric)

    X_train = dfparams.loc[:, myFeaturesNames]

    return X_train